In [59]:
# measurement

import serial
import time

ser = serial.Serial('COM8', 9600)
time.sleep(2)

measurements = [] # 측정한 전체 데이터
current_data = [] # 측정할 때 사용할 딕셔너리
measurement_started = False
saving = False
try:
    while True:
        if ser.inWaiting() > 0:

            line = ser.readline().decode('utf-8').rstrip() # 데이터 읽기

            if "Arduino" in line:
                print(line)
            
            if "Stop" in line:
                print(line)
                ser.close()
                break
            
            if "Start measurement" in line:
                print(line)
                measurement_started = True
                current_measurement = {"time": time.time(), "pin1": [], "pin2": []}  # 새로운 측정을 위해 초기화
            elif "End measurement" in line:
                print(line)
                print(f"measurement for {time.time() - current_measurement['time']}")
                measurement_started = False
                current_measurement["time"] = time.time() - current_measurement["time"]  # 측정 시간 계산
                measurements.append(current_measurement)  # 현재 측정 데이터를 전체 리스트에 추가
                print(measurements)
            elif measurement_started:
                if line.startswith("pin1: "):
                    value = int(line.split(": ")[1])
                    current_measurement["pin1"].append(value)
                elif line.startswith("pin2: "):
                    value = int(line.split(": ")[1])
                    current_measurement["pin2"].append(value)
            
except KeyboardInterrupt:
    ser.close()
    print("Serial port closed.")
    print("Collected data:", measurements)
            

--------Arduino ON--------
Start measurement
End measurement. (0.27sec)
measurement for 0.5799999237060547
[{'time': 0.5799999237060547, 'pin1': [], 'pin2': []}]
Stop


In [49]:
# 예외 떴을 때 사용 (시리얼 통신 끊기)
ser.close()

In [55]:
# initializing CustomDataset
import torch
from torch.utils.data import Dataset, ConcatDataset, DataLoader
file_name = 'potato'

class CustomDataset(Dataset):
    def __init__(self, data_list):
        self.data = data_list  # 이제 데이터는 딕셔너리의 리스트입니다.

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_point = self.data[idx]  # 인덱스를 기반으로 데이터 포인트에 접근합니다.
        # 데이터를 텐서로 변환하고 필요한 전처리를 수행
        x = torch.tensor(data_point['time'], dtype=torch.float)
        y = torch.tensor(data_point['pin1'], dtype=torch.int64)  # 리스트 형태의 데이터도 텐서로 변환 가능합니다.
        z = torch.tensor(data_point['pin2'], dtype=torch.int64)
        return x, y, z

In [53]:
# save Dataset
new_dataSet = CustomDataset(measurements)

# 기존 데이터셋이 있으면 로드, 없으면 새 데이터셋을 생성
try:
    existing_dataSet = torch.load(file_name + '.pth')
    combined_dataSet = ConcatDataset([existing_dataSet, new_dataSet])
except FileNotFoundError:
    combined_dataSet = new_dataSet

# 수정된 전체 데이터셋 저장
torch.save(combined_dataSet, file_name + '.pth')


In [60]:
# load Dataset
from torch.nn.utils.rnn import pad_sequence

loaded_dataset = torch.load(file_name + '.pth')

def my_collate_fn(batch):
    # 각 컬럼에 대해 분리하여 저장합니다.
    time, pin1, pin2 = zip(*batch)
    
    # 'time' 데이터는 이미 텐서이므로 스택을 사용하여 배치 차원을 추가합니다.
    time_tensor = torch.stack(time)
    
    # 'pin1'과 'pin2'는 서로 다른 길이를 가질 수 있으므로 패딩이 필요합니다.
    pin1_padded = pad_sequence(pin1, batch_first=True, padding_value=0)
    pin2_padded = pad_sequence(pin2, batch_first=True, padding_value=0)
    
    return time_tensor, pin1_padded, pin2_padded

# DataLoader에 로드된 데이터셋 사용
dataloader = DataLoader(loaded_dataset, batch_size=100, shuffle=True, collate_fn=my_collate_fn)

for time, pin1, pin2 in dataloader:
    print("Time:", time)
    print("Pin1:", pin1)
    print("Pin2:", pin2)


Time: tensor([2.9839, 3.4804, 3.2315, 3.2315, 2.9839])
Pin1: tensor([[ 96,  92, 149,  33,  22,  10, 584, 590, 279, 141, 101,  28,   5, 272,
         200, 112,  58,  27,  10, 444, 492, 279, 101,  44, 269,  65,  70,  30,
          15, 167, 558, 284, 150,  79,  37,   8, 252, 131,  52,  45,   6, 261,
          46, 229, 150,  36,  29,  15,  15,   7,   5,  16,   8,   8,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 65, 152,  71,  91, 121,  19,  19,   8,  10,  10,   8,   8,  86,  13,
           9,  37,  15,  58,  43,  10,   7,  19,  17,  18,  13,  11,  13,  62,
         125,  81,  41, 103,  68,  88, 110,  53,  52,  61,  36,  20,  12,   9,
          11,   8,   9,  10,   9,   9,   8,   8,  10,   8,   7,   8,   6,   8,
           6,   6,   7,   6,   7,   6,   6,   5,   5,   5,   5,   5],
        [ 38,  12,  12,  15,  22,   9,   6,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         